# Welcome to TapeAgents for Bedrock Claude LLM!


**TapeAgents** is a framework that leverages a structured, replayable log (**Tape**) of the agent session to facilitate all stages of the LLM Agent development lifecycle. In TapeAgents, the agent reasons by processing the tape and the LLM output to produce new thoughts, actions, control flow steps and append them to the tape. The environment then reacts to the agent’s actions by likewise appending observation steps to the tape.

In this tutorial, you will learn:
- how to create TapeAgents using the low-level API
- run and resume TapeAgents
- have one TapeAgent reuse another TapeAgent's tape as training data

In upcoming versions of this tutorial, you will also learn: 
- how to make a team TapeAgent with subagents
- how to build TapeAgents using available high-level APIs
- how to build a TapeAgent that streams partial steps

Other tutorials and examples will cover:
- code execution and browser use
- finetuning
- the TapeAgents apps (Studio and Browser)

# Setup
We're assuming that you already installed the project through the `make setup` or the jupyter notebook is running in the context of the project. If not, please refer to the [README](README.md) for more detailed instructions.

In [1]:
# Now set the BEDROCK_CLAUDE_API_KEY environment variable to your API key.

import os

if "BEDROCK_CLAUDE_API_KEY" not in os.environ:
    os.environ["BEDROCK_CLAUDE_API_KEY"] = "<your-api-key>"
    # os.environ["BEDROCK_CLAUDE_ORGANIZATION"] = "" # optional
today = "2025-06-10"  # fixed date for reproducible tests


# If you prefer to skip the Bedrock Claude setup and not make any LLM calls, you can use ones from the cache.
# it will work instead of the real LLM fine as long as the prompts are not changed.
# Uncomment the following lines to use the cache:
#
# from tapeagents import llms
# import os
# llm_cache_path = "tests/res/intro_notebook/tapedata.sqlite"
# if not os.path.exists(llm_cache_path):
#     llm_cache_path = f"../{llm_cache_path}"
# assert os.path.exists(llm_cache_path)
# llms._REPLAY_SQLITE = llm_cache_path

# 1. Your first TapeAgent

In this section, we will build the simplest possible "hello world" agent. We will then go through all the new concepts that you need to know to understand the code. This section is quite long, but with the solid foundation you acquire here other TapeAgent tutorials will be easy to process.

Without further ado, here's the code!

In [2]:
from tapeagents.agent import Agent, Node
from tapeagents.core import Prompt, SetNextNode
from tapeagents.dialog_tape import AssistantStep, DialogTape, UserStep
from tapeagents.llms import LiteLLM, LLMStream
from tapeagents.prompting import tape_to_messages

llm = LiteLLM(model_name="bedrock-claude-mini")


class MainNode(Node):
    name: str = "main"

    def make_prompt(self, agent: Agent, tape: DialogTape) -> Prompt:
        # Render the whole tape into the prompt, each step is converted to message
        return Prompt(messages=tape_to_messages(tape))

    def generate_steps(self, agent: Agent, tape: DialogTape, llm_stream: LLMStream):
        yield AssistantStep(content=llm_stream.get_text())  # Generate new step from the LLM output stream.
        yield SetNextNode(next_node="main")  # Which node to execute next, more on that later


agent = Agent[DialogTape].create(llm, nodes=[MainNode()])
start_tape = DialogTape(steps=[UserStep(content="Tell me about Vulcan in 3 sentences")])
final_tape = agent.run(start_tape).get_final_tape()  # agent will start executing the first node
print(f"Final tape: {final_tape.model_dump_json(indent=2)}")

/Users/oleh.shliazhko/TapeAgents/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Final tape: {
  "metadata": {
    "id": "d0d10c89-65b7-4a3c-993b-abc63e61d9b7",
    "parent_id": "8ebf62cb-64ed-46d6-b407-df9c80283be9",
    "author": "Agent",
    "author_tape_id": null,
    "n_added_steps": 2,
    "error": null,
    "result": {}
  },
  "context": null,
  "steps": [
    {
      "metadata": {
        "id": "421b7b7c-cc61-45e7-8b77-b1fc894dfe2c",
        "prompt_id": "",
        "node": "",
        "agent": "",
        "llm": "",
        "other": {}
      },
      "kind": "user",
      "content": "Tell me about Vulcan in 3 sentences"
    },
    {
      "metadata": {
        "id": "dab90836-249a-4e5b-b0bf-d565de8529c0",
        "prompt_id": "e85f2f34-57eb-4514-a842-c2f12d673726",
        "node": "main",
        "agent": "Agent",
        "llm": "default",
        "other": {}
      },
      "kind": "assistant",
      "content": "Vulcan is a fictional planet in the \"Star Trek\" universe, known as the home of the Vulcan species, including the iconic character Spock. The Vul

Now let's learn about tapes, steps, prompts, llm streams, nodes and agents.

### Tape

The fundamental concept of the TapeAgents is the `Tape`, a comprehensive semantic level log of the agent's session. A `Tape` contains a context and a sequence of `Step` objects. As you can see, a TapeAgent runs by adding steps (such as `UserStep` or `AssistantStep`) to the _tape_. This example uses the `DialogTape` tape, which is a basic tape for user-assistant conversations. Let's see what are the possible steps in a `DialogTape`.

In [3]:
# We use Python generics to instantiate many different Tape types by
# specifying different Context and Step types. In the output of this cell,
# look at Union[UserStep, AssistantStep, ...]
# for the list of possible step types in the DialogTape.
DialogTape

tapeagents.core.Tape[Union[DialogContext, NoneType], Union[UserStep, ToolResult, SystemStep, AssistantThought, SetNextNode, Pass, Call, Respond, FinalStep, AssistantStep, ToolCalls]]

Some of these steps should be familiar to you. `UserStep`, `AssistantStep`, `SystemStep` and `ToolResult` correspond to `role=user`, `role=assistant`, `role=system` and `role=tool` LLM API messages respectively. `ToolCalls` and `AssistantThought` correspond to assistant messages where the LLM requests a tool call or produces an intermediate thought that is not meant to be shown to the user. `SetNextNode` and `Pass` are TapeAgent's internal step to control which node it should run at the next iteration (more on this below).

### Prompt format; LLMs

We use the industry-standard "chat.completions" prompt format in TapeAgents: a list of user/assistant/system/tool messages plus tool schemas.

In [4]:
# Almost all classes in TapeAgents are Pydantic base models.
# This allows easy validation, serialization and instrospection. For example,
# here we are able to list all the fields in the Prompt model.
Prompt.model_fields

{'id': FieldInfo(annotation=str, required=False, default_factory=<lambda>),
 'tools': FieldInfo(annotation=Union[list[dict], NoneType], required=False, default=None),
 'messages': FieldInfo(annotation=list[dict], required=False, default_factory=list),
 'token_ids': FieldInfo(annotation=list[int], required=False, default_factory=list)}

The LLMs in TapeAgent take `Prompt` and return an `LLMStream` object. The `LLMStream` object can be used both to fast-forward to the complete response text and to stream partial outputs step by step.

In [5]:
llm_stream = LiteLLM(model_name="bedrock-claude-mini-2025-06-10", stream=True)

# Streaming
prompt = Prompt(messages=[{"role": "user", "content": "Write hello world in Java"}])
for event in llm_stream:
    print(event.get_text(), end="")

Certainly! Here’s a simple example of a "

Hello, World!" program in Java:

```java
public class Hello

World {
    public static void main(String[] args) {


        System.out.println("Hello, World!");
    }
}
```



To run this program:

1. Save the code in a file named `Hello

...

 the directory where the file is saved.
4.

 Compile the code using the command: `javac HelloWorld.java`
5.

 Run the compiled program using the command: `java HelloWorld`

You should see

 the output:

```
Hello, World!
```None
------------------------------


Certainly! Here is a simple "Hello, World!" program written in C:

```c
#include <stdio.h>

int main() {
    printf("Hello, World!\n");
    return 0;
}
```

### Explanation:
- `#include <stdio.h>`: This line includes the standard input-output library, which is necessary for using the `printf` function.
- `int main()`: This defines the main function, which is the entry point of the program.
- `printf("Hello, World!\n");`: This line prints "Hello, World!" followed by a newline character to the standard output.
- `return 0;`: This indicates that the program has finished executing successfully.

To compile and run this code:
1. Save it in a file named `hello.c`.
2. Open a terminal and navigate to the directory where you saved the file.
3. Compile the program using a C compiler, like `gcc`:
   ```
   gcc hello.c -o hello
   ```
4. Run the compiled program:
   ```
   ./hello
   ```

You should see `Hello, World!` printed on the screen.
